[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/louistrue/learn-ifc-bfh25-C/blob/main/BFH-25-FireRating-Dashboard-C.ipynb)

# Fire Rating Dashboard

Dieses Notebook erstellt ein interaktives Dashboard zur Visualisierung von Brandschutzklassifikationen (Fire Ratings) aus IFC-Modellen.

**Was Sie lernen:**
- IFC-Dateien laden und Fire Rating Daten extrahieren
- Property Sets (Psets) durchsuchen und analysieren
- Interaktive Dashboards mit Tabs erstellen
- Brandschutz-Compliance visualisieren

## 1. Abhängigkeiten installieren

Installiert alle benötigten Python-Pakete.

In [ ]:
%pip install ifcopenshell pandas plotly dash ipywidgets

## 2. IFC-Modell laden

**WICHTIG:** Passen Sie die folgenden Variablen an Ihre Gruppe an:

**Gruppe A:**
```python
github_url = 'https://raw.githubusercontent.com/louistrue/learn-ifc-bfh25-A/main/Modelle/BFH-25/03_BIMcollab_Example_STR.ifc'
local_filename = '03_BIMcollab_Example_STR.ifc'
```

**Gruppe B:**
```python
github_url = 'https://raw.githubusercontent.com/louistrue/learn-ifc-bfh25-B/main/Modelle/BFH-25/04_CT.ifc'
local_filename = '04_CT.ifc'
```

**Gruppe C:**
```python
github_url = 'https://raw.githubusercontent.com/louistrue/learn-ifc-bfh25-C/main/Modelle/BFH-25/01_Snowdon_Towers_Sample_Structural.ifc'
local_filename = '01_Snowdon_Towers_Sample_Structural.ifc'
```

**Gruppe D:**
```python
github_url = 'https://raw.githubusercontent.com/louistrue/learn-ifc-bfh25-D/main/Modelle/BFH-25/02_BIMcollab_Example_ARC.ifc'
local_filename = '02_BIMcollab_Example_ARC.ifc'
```

In [ ]:
import urllib.request
import ifcopenshell

# Gruppe C - Snowdon Towers Structural Model
github_url = 'https://raw.githubusercontent.com/louistrue/learn-ifc-bfh25-C/main/Modelle/BFH-25/01_Snowdon_Towers_Sample_Structural.ifc'
local_filename = '01_Snowdon_Towers_Sample_Structural.ifc'

print(f"📥 Downloading IFC file from GitHub...")
urllib.request.urlretrieve(github_url, local_filename)
print(f"✅ Downloaded successfully: {local_filename}")

print(f"\n📂 Loading IFC model...")
model = ifcopenshell.open(local_filename)
print(f"✅ Model loaded successfully!")
print(f"📊 Schema: {model.schema}")
print(f"🏗️ Total elements: {len(model.by_type('IfcElement'))}")

## 3. Fire Rating Daten extrahieren

Durchsucht alle Elemente nach Fire Rating Informationen in den Property Sets.

In [ ]:
import ifcopenshell.util.element as uel
import pandas as pd

print("🔍 Extracting Fire Rating data from IFC model...\n")

element_data = []
fire_rating_properties_found = set()  # Track which property names we find

for element in model.by_type('IfcElement'):
    element_id = element.GlobalId
    element_type = element.is_a()
    element_name = getattr(element, 'Name', None) or 'Unnamed'
    
    # Property Sets extrahieren
    psets = uel.get_psets(element)
    
    # Nach Fire Rating Property suchen
    fire_rating = None
    fire_rating_pset = None
    
    # Mögliche Property-Namen für Fire Rating
    possible_names = [
        'FireRating', 'fire_rating', 'FireRatingName', 'FireResistance',
        'FireRating.', 'Feuerwiderstand', 'Brandschutzklasse', 'LoadBearing'
    ]
    
    for pset_name, properties in psets.items():
        if properties and isinstance(properties, dict):
            for prop_name, prop_value in properties.items():
                # Suche nach Fire Rating Properties
                if any(possible_name.lower() in prop_name.lower() for possible_name in possible_names):
                    fire_rating = prop_value
                    fire_rating_pset = pset_name
                    fire_rating_properties_found.add(f"{pset_name}.{prop_name}")
                    break
        if fire_rating:
            break
    
    # Geschoss finden
    storey = 'Not assigned'
    for rel in model.by_type('IfcRelContainedInSpatialStructure'):
        if element in rel.RelatedElements:
            if rel.RelatingStructure and rel.RelatingStructure.is_a('IfcBuildingStorey'):
                storey = rel.RelatingStructure.LongName or rel.RelatingStructure.Name
                break
    
    element_data.append({
        'ElementId': element_id,
        'ElementType': element_type,
        'ElementName': element_name,
        'FireRating': str(fire_rating) if fire_rating is not None else 'Nicht klassifiziert',
        'PropertySet': fire_rating_pset or 'N/A',
        'Storey': storey
    })

# DataFrame erstellen
df = pd.DataFrame(element_data)

# Statistiken ausgeben
print(f"✅ Data extraction complete!\n")
print(f"📊 Total elements: {len(df)}")
print(f"🔥 Elements with Fire Rating: {len(df[df['FireRating'] != 'Nicht klassifiziert'])}")
print(f"❌ Elements without Fire Rating: {len(df[df['FireRating'] == 'Nicht klassifiziert'])}")
print(f"\n🏷️ Fire Rating properties found in model:")
for prop in sorted(fire_rating_properties_found):
    print(f"   - {prop}")

print(f"\n🔥 Unique Fire Ratings found:")
unique_ratings = df[df['FireRating'] != 'Nicht klassifiziert']['FireRating'].unique()
for rating in sorted(unique_ratings):
    count = len(df[df['FireRating'] == rating])
    print(f"   - {rating}: {count} elements")

# Preview der Daten
print(f"\n📋 Data preview:")
df.head(10)

## 4. Daten für Visualisierung vorbereiten

Erstellt aggregierte DataFrames für verschiedene Dashboard-Ansichten.

In [ ]:
print("📊 Preparing data for visualization...\n")

# Fire Rating Verteilung (Gesamtübersicht)
fire_rating_counts = (
    df.groupby('FireRating')['ElementId']
    .nunique()
    .reset_index(name='Count')
    .sort_values('Count', ascending=False)
)

# Fire Rating nach Elementtyp
fire_rating_by_type = (
    df.groupby(['FireRating', 'ElementType'])['ElementId']
    .nunique()
    .reset_index(name='Count')
    .sort_values('Count', ascending=False)
)

# Fire Rating nach Geschoss
fire_rating_by_storey = (
    df.groupby(['Storey', 'FireRating'])['ElementId']
    .nunique()
    .reset_index(name='Count')
    .sort_values(['Storey', 'Count'], ascending=[True, False])
)

# Elementtyp nach Fire Rating (für Stacked Bar)
element_type_by_rating = (
    df.groupby(['ElementType', 'FireRating'])['ElementId']
    .nunique()
    .reset_index(name='Count')
    .sort_values('Count', ascending=False)
)

# Nicht klassifizierte Elemente nach Typ
unclassified = df[df['FireRating'] == 'Nicht klassifiziert'].copy()
unclassified_by_type = (
    unclassified.groupby('ElementType')['ElementId']
    .nunique()
    .reset_index(name='Count')
    .sort_values('Count', ascending=False)
)

print("✅ Data preparation complete!\n")
print(f"📈 Created {len(fire_rating_counts)} fire rating categories")
print(f"🏗️ Analyzed {len(element_type_by_rating)} element type combinations")
print(f"🏢 Covered {len(fire_rating_by_storey['Storey'].unique())} building storeys")
print(f"⚠️ Found {len(unclassified)} unclassified elements")

## 5. Interaktives Dashboard erstellen

Erstellt ein Dash-Dashboard mit mehreren Tabs zur Visualisierung der Fire Rating Daten.

In [ ]:
from dash import Dash, dcc, html, dash_table, Input, Output
import plotly.express as px
import plotly.graph_objects as go

print("🎨 Creating interactive dashboard...\n")

app = Dash(__name__)

# Funktion zum Erstellen der Diagramme mit optionaler Filterung
def create_figures(include_unclassified=True):
    """Erstellt alle Dashboard-Diagramme mit optionaler Filterung."""
    
    # Daten filtern wenn nötig
    if include_unclassified:
        filtered_df = df
    else:
        filtered_df = df[df['FireRating'] != 'Nicht klassifiziert']
    
    # Aggregationen mit gefilterten Daten neu erstellen
    filtered_fire_rating_counts = (
        filtered_df.groupby('FireRating')['ElementId']
        .nunique()
        .reset_index(name='Count')
        .sort_values('Count', ascending=False)
    )
    
    filtered_fire_rating_by_type = (
        filtered_df.groupby(['FireRating', 'ElementType'])['ElementId']
        .nunique()
        .reset_index(name='Count')
        .sort_values('Count', ascending=False)
    )
    
    filtered_fire_rating_by_storey = (
        filtered_df.groupby(['Storey', 'FireRating'])['ElementId']
        .nunique()
        .reset_index(name='Count')
        .sort_values(['Storey', 'Count'], ascending=[True, False])
    )
    
    filtered_element_type_by_rating = (
        filtered_df.groupby(['ElementType', 'FireRating'])['ElementId']
        .nunique()
        .reset_index(name='Count')
        .sort_values('Count', ascending=False)
    )
    
    # 1. Fire Rating Übersicht (Bar Chart)
    fig_overview = px.bar(
        filtered_fire_rating_counts,
        x='FireRating',
        y='Count',
        title='Fire Rating Verteilung',
        text_auto='.0f',
        color='Count',
        color_continuous_scale='Reds'
    )
    fig_overview.update_layout(
        xaxis_title='Fire Rating',
        yaxis_title='Anzahl Elemente',
        coloraxis_showscale=False
    )
    
    # 2. Fire Rating nach Elementtyp (Stacked Bar)
    fig_by_type = px.bar(
        filtered_element_type_by_rating,
        x='ElementType',
        y='Count',
        color='FireRating',
        title='Fire Rating nach Elementtyp',
        text_auto='.0f'
    )
    fig_by_type.update_layout(
        xaxis_title='Elementtyp',
        yaxis_title='Anzahl Elemente',
        legend_title='Fire Rating',
        xaxis_tickangle=-45
    )
    
    # 3. Fire Rating nach Geschoss (Stacked Bar)
    fig_by_storey = px.bar(
        filtered_fire_rating_by_storey,
        x='Storey',
        y='Count',
        color='FireRating',
        title='Fire Rating nach Geschoss',
        text_auto='.0f'
    )
    fig_by_storey.update_layout(
        xaxis_title='Geschoss',
        yaxis_title='Anzahl Elemente',
        legend_title='Fire Rating'
    )
    
    # 4. Heatmap: Fire Rating vs Elementtyp
    if len(filtered_fire_rating_by_type) > 0:
        # Pivot für Heatmap vorbereiten
        heatmap_data = filtered_fire_rating_by_type.pivot_table(
            index='FireRating',
            columns='ElementType',
            values='Count',
            fill_value=0
        )
        
        fig_heatmap = px.imshow(
            heatmap_data,
            title='Fire Rating vs Elementtyp (Heatmap)',
            labels=dict(x="Elementtyp", y="Fire Rating", color="Anzahl"),
            color_continuous_scale='YlOrRd',
            aspect='auto'
        )
        fig_heatmap.update_xaxes(side='bottom', tickangle=-45)
    else:
        fig_heatmap = go.Figure()
        fig_heatmap.add_annotation(
            text="Keine Daten für Heatmap verfügbar",
            xref="paper", yref="paper",
            x=0.5, y=0.5, showarrow=False
        )
    
    return {
        'overview': fig_overview,
        'by_type': fig_by_type,
        'by_storey': fig_by_storey,
        'heatmap': fig_heatmap,
        'rating_counts': filtered_fire_rating_counts
    }

# Initiale Diagramme erstellen
initial_figures = create_figures(include_unclassified=True)

# Diagramm für nicht klassifizierte Elemente
if len(unclassified_by_type) > 0:
    fig_unclassified = px.bar(
        unclassified_by_type,
        x='ElementType',
        y='Count',
        title=f'Nicht klassifizierte Elemente nach Typ ({len(unclassified)} total)',
        text_auto='.0f',
        color='Count',
        color_continuous_scale='Oranges'
    )
    fig_unclassified.update_layout(
        xaxis_title='Elementtyp',
        yaxis_title='Anzahl',
        coloraxis_showscale=False,
        xaxis_tickangle=-45
    )
else:
    fig_unclassified = None

# Dashboard Layout
tabs_list = [
    dcc.Tab(
        label='📊 Übersicht',
        children=[
            html.Div([
                html.H3('Fire Rating Gesamtverteilung'),
                html.P(
                    'Wie viele Elemente haben welche Brandschutzklassifikation? '
                    'Diese Übersicht zeigt die Verteilung aller Fire Ratings im Gebäude.'
                ),
                dcc.Graph(id='fig-overview', figure=initial_figures['overview']),
                html.Hr(),
                html.H3('Fire Rating Statistik'),
                dash_table.DataTable(
                    id='table-overview',
                    data=initial_figures['rating_counts'].to_dict('records'),
                    columns=[{'name': col, 'id': col} for col in initial_figures['rating_counts'].columns],
                    style_table={'maxHeight': '400px', 'overflowY': 'auto'},
                    style_cell={'padding': '0.5rem', 'textAlign': 'left'},
                    style_header={'fontWeight': 'bold', 'backgroundColor': '#f0f0f0'},
                    style_data_conditional=[
                        {'if': {'row_index': 'odd'}, 'backgroundColor': '#f9f9f9'}
                    ]
                )
            ], style={'padding': '20px'})
        ]
    ),
    dcc.Tab(
        label='🏗️ Nach Elementtyp',
        children=[
            html.Div([
                html.H3('Fire Rating nach Bauteiltyp'),
                html.P(
                    'Welche Elementtypen haben welche Fire Ratings? '
                    'Gestapelte Balken zeigen die Fire Rating-Verteilung pro Bauteilkategorie.'
                ),
                dcc.Graph(id='fig-by-type', figure=initial_figures['by_type'])
            ], style={'padding': '20px'})
        ]
    ),
    dcc.Tab(
        label='🏢 Nach Geschoss',
        children=[
            html.Div([
                html.H3('Fire Rating pro Geschoss'),
                html.P(
                    'Wie sind die Fire Ratings über die Geschosse verteilt? '
                    'Diese Ansicht hilft, Brandschutzanforderungen pro Ebene zu verstehen.'
                ),
                dcc.Graph(id='fig-by-storey', figure=initial_figures['by_storey'])
            ], style={'padding': '20px'})
        ]
    ),
    dcc.Tab(
        label='🔥 Heatmap',
        children=[
            html.Div([
                html.H3('Fire Rating Heatmap'),
                html.P(
                    'Welche Kombination aus Fire Rating und Elementtyp ist am häufigsten? '
                    'Die Heatmap zeigt Muster und Schwerpunkte auf einen Blick.'
                ),
                dcc.Graph(id='fig-heatmap', figure=initial_figures['heatmap'])
            ], style={'padding': '20px'})
        ]
    )
]

# Tab für nicht klassifizierte Elemente hinzufügen (falls vorhanden)
if len(unclassified_by_type) > 0:
    tabs_list.append(
        dcc.Tab(
            label=f'⚠️ Nicht klassifiziert ({len(unclassified)})',
            children=[
                html.Div([
                    html.H3('Elemente ohne Fire Rating'),
                    html.P([
                        'Diese Elemente haben keine Fire Rating Klassifikation in ihren Property Sets. '
                        'Dies könnte auf fehlende Daten im Modell hinweisen und sollte überprüft werden.'
                    ]),
                    dcc.Graph(figure=fig_unclassified),
                    html.Hr(),
                    html.H3('Details zu nicht klassifizierten Elementen'),
                    dash_table.DataTable(
                        data=unclassified_by_type.to_dict('records'),
                        columns=[{'name': col, 'id': col} for col in unclassified_by_type.columns],
                        style_table={'maxHeight': '400px', 'overflowY': 'auto'},
                        style_cell={'padding': '0.5rem', 'textAlign': 'left'},
                        style_header={'fontWeight': 'bold', 'backgroundColor': '#fff3cd'},
                        style_data_conditional=[
                            {'if': {'row_index': 'odd'}, 'backgroundColor': '#fffbf0'}
                        ]
                    )
                ], style={'padding': '20px'})
            ]
        )
    )

app.layout = html.Div([
    html.Div([
        html.H1('🔥 Fire Rating Dashboard', style={'color': '#d32f2f'}),
        html.P(
            'Interaktive Visualisierung der Brandschutzklassifikationen aus dem IFC-Modell. '
            'Nutzen Sie die Tabs, um verschiedene Ansichten der Fire Rating-Daten zu erkunden.',
            style={'fontSize': '16px', 'color': '#666'}
        ),
    ], style={'padding': '20px', 'backgroundColor': '#f5f5f5', 'borderBottom': '3px solid #d32f2f'}),
    
    # Toggle für nicht klassifizierte Elemente
    html.Div([
        html.Label('Filter-Optionen:', style={'fontWeight': 'bold', 'marginRight': '10px'}),
        dcc.Checklist(
            id='include-unclassified-toggle',
            options=[
                {'label': ' Nicht klassifizierte Elemente in Visualisierungen einbeziehen', 'value': 'include'}
            ],
            value=['include'],
            style={'display': 'inline-block'}
        ),
    ], style={
        'backgroundColor': '#e3f2fd',
        'padding': '15px',
        'margin': '20px',
        'borderRadius': '5px',
        'border': '1px solid #90caf9'
    }),
    
    dcc.Tabs(tabs_list, style={'padding': '0 20px'}),
    
    html.Div([
        html.Hr(),
        html.P(
            f'📊 Modell: {local_filename} | '
            f'Elemente gesamt: {len(df)} | '
            f'Mit Fire Rating: {len(df[df["FireRating"] != "Nicht klassifiziert"])} | '
            f'Ohne Fire Rating: {len(df[df["FireRating"] == "Nicht klassifiziert"])}',
            style={'textAlign': 'center', 'color': '#999', 'fontSize': '14px'}
        )
    ], style={'padding': '20px'})
])

# Callback zum Aktualisieren der Diagramme basierend auf dem Toggle
@app.callback(
    [
        Output('fig-overview', 'figure'),
        Output('fig-by-type', 'figure'),
        Output('fig-by-storey', 'figure'),
        Output('fig-heatmap', 'figure'),
        Output('table-overview', 'data')
    ],
    Input('include-unclassified-toggle', 'value')
)
def update_dashboard(toggle_value):
    """Aktualisiert alle Diagramme basierend auf dem Toggle-Status."""
    include_unclassified = 'include' in toggle_value
    figures = create_figures(include_unclassified=include_unclassified)
    
    return (
        figures['overview'],
        figures['by_type'],
        figures['by_storey'],
        figures['heatmap'],
        figures['rating_counts'].to_dict('records')
    )

print("✅ Dashboard created successfully!")
print("🚀 Starting dashboard...\n")

# Dashboard im Jupyter Modus starten
app.run(jupyter_mode='inline', height=900, port=8050)

## 6. Nächste Schritte

### Dashboard erweitern:
- Dropdown-Filter für spezifische Geschosse oder Fire Ratings hinzufügen
- Daten nach CSV exportieren für Berichte
- Compliance-Metriken berechnen (z.B. % der Elemente mit korrekter Klassifikation)

### Weitere Modelle testen:
- Eigene IFC-Dateien hochladen und analysieren
- Verschiedene Gebäude oder Design-Versionen vergleichen
- Datenqualität und Vollständigkeit analysieren

### Code auf GitHub hochladen:
1. **In Colab:** File → Save a copy in GitHub
2. **Repository wählen:** `louistrue/learn-ifc-bfh25-X` (X = eure Gruppe)
3. **Dateiname:** `BFH-25-FireRating-Dashboard-X.ipynb`
4. **Commit:** "Add Fire Rating Dashboard - Group X"

## 📚 Dokumentation

- [IFCOpenShell Docs](https://docs.ifcopenshell.org/)
- [Plotly Python](https://plotly.com/python/)
- [Dash Documentation](https://dash.plotly.com/)
- [Pandas Guide](https://pandas.pydata.org/docs/)